这里我们使用别人已经编写的 deepCTR 库.来验证 din 深度学习兴趣网络 [中文知乎地址](https://zhuanlan.zhihu.com/p/53231955), 英文知乎地址 [英文知乎地址](https://zhuanlan.zhihu.com/p/53231955)

具体的依赖安装

pip install deepctr[cpu]#cpu版本

pip install deepctr[gpu]#GPU版本

对应 torch 要添加 torch
pip  install deepctr-torch

我们简单了解一下 deepCTR 的核心设计
首先在设计过程中采用模块化的设计,各个模块相互独立,基于深度学习的点击率预测模型按模型内部组件的功能可以划分成以下4个模块：输入模块，嵌入模块，特征提取模块，预测输出模块。

![模块化设计](https://pic1.zhimg.com/80/v2-392784585d6238db7f20744e8e98c5f4_1440w.jpg "Magic Gardens")


python 当中的多根工作区配置 https://code.visualstudio.com/docs/editor/multi-root-workspaces

使用 mac ```m1```  进行 python 开发的时候,需要踩坑就是 官方的 conda 并不支持 mac m1 版本,所以很多软件并不能正常运行,例如 TensorFlow pytorch 等,这里我们使用支持的m1 conda  [支持mac
_m1的 conda](https://github.com/conda-forge/miniforge/#download) ,这里安装尽可能使用 
```shell
conda install -c conda-forge tqdm
```



使用 vscode 开发需要注意两点

1.vscode 的 setting.json 配置说明,后续补充连接

```json
{
  "python.pythonPath": "$HOME/miniconda3/envs/deepctr/bin/python",
}
```
另外一个示例
```json
{
    "python.autoComplete.extraPaths": ["${workspaceFolder}/deepctr_torch"],
    "python.envFile": "${workspaceFolder}/.env",
    "python.analysis.extraPaths": [
        "${workspaceFolder}/deepctr_torch"
    ],
    "jupyter.jupyterServerType": "local",
    "terminal.integrated.env.osx": {"PYTHONPATH": "${workspaceFolder}${pathSeparator}${env:PYTHONPATH}"}
}
```

2.launcher.json 配置说明

```json 
{
    // Use IntelliSense to learn about possible attributes.
    // Hover to view descriptions of existing attributes.
    // For more information, visit: https://go.microsoft.com/fwlink/?linkid=830387
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Python: 当前文件",
            "type": "python",
            "request": "launch",
            "program": "${file}",
            "console": "integratedTerminal",
            "justMyCode": true,
            "cwd": "${fileDirname}",
            "env": {"PYTHONPATH": "${workspaceFolder}${pathSeparator}${env:PYTHONPATH}"}
        }
    ]
}
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import plotly.graph_objs as go
# 在线模式不要使用,下面画图 使用 plotly 离线模式
import plotly.offline as py
py.init_notebook_mode(connected=False)
#import chart_studio.plotly as py #在线模式不用使用,很卡
import plotly.figure_factory as ff
# 配置pandas 的默认绘图使用 plotly
pd.options.plotting.backend = 'plotly'
# 引入高级绘图模式 plotly.express ,低级别的绘图模式是 plotly.graph_objs
import plotly.express as px

In [ ]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
# 使用pandas 读取上面介绍的数据，并进行简单的缺失值填充
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.models import *
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
import torch

```shell
! \rm -vf ./data/criteo_sample.txt
! wget https://github.com/shenweichen/DeepCTR-Torch/blob/master/examples/criteo_sample.txt -O ./data/criteo_sample.txt
```

In [ ]:
# 使用pandas 读取上面介绍的数据，并进行简单的缺失值填充
data = pd.read_csv('./data/criteo_sample.txt')
# 上面的数据在：https://github.com/shenweichen/DeepCTR-Torch/blob/master/examples/criteo_sample.txt
table = ff.create_table(data.head(10))
py.iplot(table, filename='jupyter-table1')


使用 pandas+polly 注意是为了针对数据做统计分析

In [ ]:
trace1 = go.Scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16],mode='markers')
layout=go.Layout(title='Hello Plotly!!', xaxis=dict(title='x axis'), yaxis=dict(title='y axis'))
#一个 Figure 对象要包含三部分 data layout 和 frames, 其中 data 和 layout 可以是空的,后续可以动态更新
fig=go.Figure(data=[trace1],layout=layout)
#如果想查看数据可以使用 print(fig.to_json())
# 使用 fig.show() 显示图表也可以
py.iplot(fig, filename='jupyter-plot1')

下面我们继续回到 deepCTR的案例上来

In [ ]:
#特征列添加标志27 个稀疏特征, 14 个稠密特征, 我们给稀疏特征 添加 C 的标志,给稠密特征添加 I 的标志
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]
dense_features

In [ ]:
# 特征缺失值处理 稀疏值 补充-1,稠密特征缺失值处理 填充0
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

data.head(10)

In [ ]:
#这里我们需要对特征进行一些预处理，对于类别特征，我们使用LabelEncoder重新编码(或者哈希编码)，对于数值特征使用MinMaxScaler压缩到0~1之间。
# 这里我们使用的 sklearn 的 LabelEncoder 对类别特征进行编码,使用的 MinMaxScaler 讲数值特征压缩到0~1之间
for feat in sparse_features:
   lbe = LabelEncoder()
   data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])


In [ ]:
# 这里是比较关键的一步，因为我们需要对类别特征进行Embedding，所以需要告诉模型每一个特征组有多少个embbedding向量，我们通过pandas的nunique()方法统计。
# SparseFeat(特征名称,特征维度,embedding_size,embedding_name) 这里采用的是命名元组实现的
# DensFeat(特征名称,特征维度)
# 核心逻辑将 稀疏特征 和 稠密特征进行拼接 
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique()) for feat in sparse_features] + [ DenseFeat(feat, 1,) for feat in dense_features]

# 深度学习模型的特征输入 
dnn_feature_columns = fixlen_feature_columns
# 线性模型的输入
linear_feature_columns = fixlen_feature_columns

# 输入特征的名称
fixlen_feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
fixlen_feature_names[20:30]

In [ ]:

#最后，我们按照上一步生成的特征列拼接数据
# 8 2 原则拆分训练集和验证集
train, test = train_test_split(data, test_size=0.2)

train_model_input = [train[name] for name in fixlen_feature_names]
test_model_input = [test[name] for name in fixlen_feature_names]

train.head(5)

In [ ]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
   print('cuda ready...')
   device = 'cuda:0'

device

In [17]:
# 初始化模型，进行训练和预测

model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns, task='binary',
               l2_reg_embedding=1e-5, device=device)

model.compile("adagrad", "binary_crossentropy",
               metrics=["binary_crossentropy", "auc"],)
model.fit(train_model_input, train[target].values,
           batch_size=256, epochs=100, validation_split=0.2, verbose=2)

pred_ans = model.predict(test_model_input, 256)
print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

Epoch 13/100
67s - loss:  0.0539 - binary_crossentropy:  0.0539 - auc:  1.0000 - val_binary_crossentropy:  0.8821 - val_auc:  0.3864
Epoch 14/100
69s - loss:  0.0434 - binary_crossentropy:  0.0434 - auc:  1.0000 - val_binary_crossentropy:  0.9054 - val_auc:  0.3773
Epoch 15/100
68s - loss:  0.0353 - binary_crossentropy:  0.0353 - auc:  1.0000 - val_binary_crossentropy:  0.9274 - val_auc:  0.3727
Epoch 16/100
66s - loss:  0.0291 - binary_crossentropy:  0.0291 - auc:  1.0000 - val_binary_crossentropy:  0.9492 - val_auc:  0.3727
Epoch 17/100
68s - loss:  0.0243 - binary_crossentropy:  0.0243 - auc:  1.0000 - val_binary_crossentropy:  0.9710 - val_auc:  0.3773
Epoch 18/100
68s - loss:  0.0207 - binary_crossentropy:  0.0207 - auc:  1.0000 - val_binary_crossentropy:  0.9955 - val_auc:  0.3773
Epoch 19/100
67s - loss:  0.0180 - binary_crossentropy:  0.0180 - auc:  1.0000 - val_binary_crossentropy:  1.0177 - val_auc:  0.3727
Epoch 20/100
69s - loss:  0.0159 - binary_crossentropy:  0.0159 - auc

使用 summary 可视化 模型结构,以前叫 torch_summary,现在叫 torchinfo

```shel
conda install -c conda-forge torchinfo
```

In [ ]:
#使用 torch_summary 进行可视化
from torchinfo import summary
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
t = model.to(device)
summary(t, input_size=(1, len(fixlen_feature_names)))

In [ ]:
print(model)